Calculate cumulative number of cases

In [2]:
# import dependencies
import pandas as pd

In [3]:
# set province
province = "CATANDUANES"

In [ ]:
# load csv file# read from csv file
df = pd.read_csv(f"aggregated_data/{province}_old.csv")
df["Date"] = pd.to_datetime(df["Date"])

df.head()




,Unnamed: 0,Date,Municipality,Latitude,Longitude,NewCases,Deaths,Recoveries,d_cases,n
0,0,2020-06-06,SAN MIGUEL,13.640329,124.303109,1,0,0,1,1
1,6,2020-07-16,SAN MIGUEL,13.640329,124.303109,0,0,1,-1,0
2,66,2020-09-10,SAN MIGUEL,13.640329,124.303109,1,0,0,1,1
3,79,2020-09-20,SAN MIGUEL,13.640329,124.303109,0,0,1,-1,0
4,89,2020-09-24,SAN MIGUEL,13.640329,124.303109,1,0,0,1,1


In [43]:
# add new column for change in cases
df["d_cases"] = df["NewCases"] - df["Deaths"] - df["Recoveries"]

df

,Unnamed: 0,Date,Municipality,Latitude,Longitude,NewCases,Deaths,Recoveries,d_cases,n
0,0,2020-06-06,SAN MIGUEL,13.640329,124.303109,1,0,0,1,1
1,6,2020-07-16,SAN MIGUEL,13.640329,124.303109,0,0,1,-1,0
2,66,2020-09-10,SAN MIGUEL,13.640329,124.303109,1,0,0,1,1
3,79,2020-09-20,SAN MIGUEL,13.640329,124.303109,0,0,1,-1,0
4,89,2020-09-24,SAN MIGUEL,13.640329,124.303109,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...
2570,2483,2023-06-09,VIGA,13.871531,124.308730,0,0,1,-1,0
2571,2511,2023-06-28,VIGA,13.871531,124.308730,1,0,0,1,1
2572,2517,2023-07-07,VIGA,13.871531,124.308730,0,0,1,-1,0
2573,2567,2023-12-30,VIGA,13.871531,124.308730,1,0,0,1,1


In [44]:
# get min and max date
min_date = df["Date"].min()
max_date = df["Date"].max()

(min_date, max_date)

(Timestamp('2020-06-06 00:00:00'), Timestamp('2024-01-08 00:00:00'))

In [51]:
# create series of date from min to max
date_range = pd.date_range(start=min_date, end=max_date).to_frame(name="Date")

#date_range = date_range['Date'].dt.date
date_range

,Date
2020-06-06,2020-06-06
2020-06-07,2020-06-07
2020-06-08,2020-06-08
2020-06-09,2020-06-09
2020-06-10,2020-06-10
...,...
2024-01-04,2024-01-04
2024-01-05,2024-01-05
2024-01-06,2024-01-06
2024-01-07,2024-01-07


In [56]:
# calculate cumulative sum
new_df = pd.DataFrame()

mun_list = df["Municipality"].unique().sort()

print(mun_list)

for mun in df["Municipality"].unique():
    #print(mun)
    df_mun = df[df["Municipality"] == mun]

    # combine with date range
    df_mun2 = pd.DataFrame()
    df_mun2["Date"] = date_range

    #print(df_mun["Date"].dtype)

    #for date in df_mun2["Date"]:
    #    if date in df_mun["Date"]:
    #        print("test")
        #print(date)

    df_mun2 = pd.merge(df_mun2, df_mun, how="outer", on="Date")
    #df_mun2 = pd.concat([df_mun2, df_mun])
    #df_mun2 = pd.concat([df_mun2, date_range], join="outer", verify_integrity=True)

    df_mun2["Date"] = pd.to_datetime(df_mun2.Date, format='mixed')
    df_mun2 = df_mun2.sort_values(by="Date")

    df_mun2['Municipality'] = mun
    df_mun2["Longitude"] = df_mun.iloc[0]["Longitude"]
    df_mun2["Latitude"] = df_mun.iloc[0]["Latitude"]
    df_mun2 = df_mun2.fillna(0)

    df_mun2 = df_mun2.reset_index()

    df_mun2 = df_mun2.drop(['index', 'Unnamed: 0'], axis=1)
    
    # compute cumulative data
    df_mun2["n"] = df_mun2["d_cases"].cumsum()
    
    #print(df_mun2.head(10))

    print(df_mun2)

    new_df = pd.concat([new_df, df_mun2])

#df["n"] = df["d_cases"].cumsum()
new_df

None
           Date Municipality   Latitude   Longitude  NewCases  Deaths   
0    2020-06-06   SAN MIGUEL  13.640329  124.303109       1.0     0.0  \
1    2020-06-07   SAN MIGUEL  13.640329  124.303109       0.0     0.0   
2    2020-06-08   SAN MIGUEL  13.640329  124.303109       0.0     0.0   
3    2020-06-09   SAN MIGUEL  13.640329  124.303109       0.0     0.0   
4    2020-06-10   SAN MIGUEL  13.640329  124.303109       0.0     0.0   
...         ...          ...        ...         ...       ...     ...   
1307 2024-01-04   SAN MIGUEL  13.640329  124.303109       0.0     0.0   
1308 2024-01-05   SAN MIGUEL  13.640329  124.303109       0.0     0.0   
1309 2024-01-06   SAN MIGUEL  13.640329  124.303109       0.0     0.0   
1310 2024-01-07   SAN MIGUEL  13.640329  124.303109       0.0     0.0   
1311 2024-01-08   SAN MIGUEL  13.640329  124.303109       0.0     0.0   

      Recoveries  d_cases    n  
0            0.0      1.0  1.0  
1            0.0      0.0  1.0  
2            0.0   

,Date,Municipality,Latitude,Longitude,NewCases,Deaths,Recoveries,d_cases,n
0,2020-06-06,SAN MIGUEL,13.640329,124.303109,1.0,0.0,0.0,1.0,1.0
1,2020-06-07,SAN MIGUEL,13.640329,124.303109,0.0,0.0,0.0,0.0,1.0
2,2020-06-08,SAN MIGUEL,13.640329,124.303109,0.0,0.0,0.0,0.0,1.0
3,2020-06-09,SAN MIGUEL,13.640329,124.303109,0.0,0.0,0.0,0.0,1.0
4,2020-06-10,SAN MIGUEL,13.640329,124.303109,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
1307,2024-01-04,VIGA,13.871531,124.308730,0.0,0.0,0.0,0.0,1.0
1308,2024-01-05,VIGA,13.871531,124.308730,0.0,0.0,0.0,0.0,1.0
1309,2024-01-06,VIGA,13.871531,124.308730,0.0,0.0,0.0,0.0,1.0
1310,2024-01-07,VIGA,13.871531,124.308730,0.0,0.0,1.0,-1.0,0.0


In [57]:
# export to csv

new_df.to_csv(f"aggregated_data/{province}.csv")